This works on datarmor: 
qsub -I -q mpi_1 -l walltime=1:00:00
bash
conda activate /home/datawork-lops-iaocea/conda-env/pangeo-fish_1222/
cd /home/datawork-taos-s/test
ipython

In [ ]:
url_in = 'file:///home/datawork-taos-s/public/kerchunk_file/MARC_F1-MARS3D-SEINE_20180801T0000Z.nc.json'
#url_in= 'https://data-taos.ifremer.fr/kerchunk/MARC_F1-MARS3D-SEINE_20180801T0800Z.nc.json'

d= url_in
import xarray as xr
import kerchunk
backend_args = {
    "consolidated": False,
    "storage_options": {
        "fo": url_in,
#        "target_options": {
#            "compression": "zstd"
#        },
#        "target_protocol": "file"
    }}


backend_args= {
        "storage_options": {
            "fo": url_in,
        },
        "consolidated": False
    }
ds=xr.open_mfdataset("reference://", engine="zarr", backend_kwargs=backend_args)
ds.H0.min().compute()


From my PC, using the same kerchunk file, can show the metadata but can't access to data it self. (because the path is from inside HPC)

In [1]:
#url_in = 'file:///home/datawork-taos-s/public/kerchunk/MARC_F1-MARS3D-SEINE_20180801T0000Z.nc.json'
url_in= 'https://data-taos.ifremer.fr/kerchunk/MARC_F1-MARS3D-SEINE_20180801T0800Z.nc.json'

d= url_in
import xarray as xr
import kerchunk
backend_args = {
    "consolidated": False,
    "storage_options": {
        "fo": url_in,
#        "target_options": {
#            "compression": "zstd"
#        },
#        "target_protocol": "file"
    }}


backend_args= {
        "storage_options": {
            "fo": url_in,
        },
        "consolidated": False
    }
ds=xr.open_mfdataset("reference://", engine="zarr", backend_kwargs=backend_args)
ds.H0.min()#.compute()

<xarray.DataArray 'H0' ()>
dask.array<_nanmin_skip-aggregate, shape=(), dtype=float32, chunksize=(), chunktype=numpy.ndarray>

So it needs translation. 
Follwoing cell is the functions we need

In [2]:
import fsspec
import ujson

def match_keys(mapping, value):
    for k in mapping:
        if k in value: 
            return k
        
    raise ValueError(f"could not find {value} in mapping") 
    
def match_in_keys(mapping,value):
    try:
        match_keys(mapping,value)
        return True
    except ValueError:
        return False    

def rename_target(refs, renames):
    #from kerchunk.utils import conslidate
    print('in rename_target')
    """Utility to change URLs in a reference set in a predictable way

    For reference sets including templates, this is more easily done by
    using template overrides at access time; but rewriting the references
    and saving a new file means not having to do that every time.

    Parameters
    ----------
    refs: dict
        Reference set
    renames: dict[str, str]
        Mapping from the old URL (including protocol, if this is how they appear
        in the original) to new URL

    Returns
    -------
    dict: the altered reference set, which can be saved
    """
    fs = fsspec.filesystem("reference", fo=refs)  # to produce normalised refs
    refs = fs.references
    out = {}
    for k, v in refs.items():
        if isinstance(v, list) and v[0] in renames:
            out[k] = [renames[v[0]]] + v[1:]
        elif isinstance(v, list) and match_in_keys(renames, v[0]) :
            url = v[0]
            key = match_keys(renames, url)
            new_url = url.replace(key, renames[key])
            out[k] = [new_url] + v[1:]
            #print(new_url)
        else:
            out[k] = v
        #    print('boo')
    return consolidate(out)

def rename_target_files(
    url_in, renames, url_out=None, storage_options_in=None, storage_options_out=None):
    print('in rename_target_files')

    """Perform URL renames on a reference set - read and write from JSON

    Parameters
    ----------
    url_in: str
        Original JSON reference set
    renames: dict
        URL renamings to perform (see ``renate_target``)
    url_out: str | None
        Where to write to. If None, overwrites original
    storage_options_in: dict | None
        passed to fsspec for opening url_in
    storage_options_out: dict | None
        passed to fsspec for opening url_out. If None, storage_options_in is used.

    Returns
    -------
    None
    """
    with fsspec.open(url_in, **(storage_options_in or {})) as f:
        old = ujson.load(f)
    new = rename_target(old, renames)
    if url_out is None:
        url_out = url_in
    if storage_options_out is None:
        storage_options_out = storage_options_in
    with fsspec.open(url_out, mode="wt", **(storage_options_out or {})) as f:
        ujson.dump(new, f)

def consolidate(refs):
    """Turn raw references into output"""
    out = {}
    for k, v in refs.items():
        if isinstance(v, bytes):
            try:
                # easiest way to test if data is ascii
                out[k] = v.decode("ascii")
            except UnicodeDecodeError:
                out[k] = (b"base64:" + base64.b64encode(v)).decode()
        else:
            out[k] = v
    return {"version": 1, "refs": out}

We translate url_in to url_out using renames. This rename can corresponds to all the marc file. 
First test without any compression suceeded.  

In [7]:
url_in= 'https://data-taos.ifremer.fr/kerchunk_file/MARC_F1-MARS3D-SEINE_20180801T0800Z.nc.json'
url_out = 'file:///Users/todaka/python/git/auto-kerchunk/test.json'


url_out = 'file:///home/datawork-taos-s/public/kerchunk_https/test.json'



renames={'file:///home/ref-marc/':'https://data-dataref.ifremer.fr/marc/'} 

storage_options_in= {
#        "fo": d,
#        "target_options": {
#            "compression": "zstd"
#        },
        "target_protocol": "https"
    }

storage_options_out= {
#        "fo": d,
#        "target_options": {
#            "compression": "zstd"
#        },
        "target_protocol": "file"
    }

rename_target_files(
    url_in, renames, url_out, storage_options_in=None
    , storage_options_out=None)

in rename_target_files
in rename_target
https://data-dataref.ifremer.fr/marc/f1_e2500_agrif/MARC_F1-MARS3D-SEINE/best_estimate/2018/MARC_F1-MARS3D-SEINE_20180801T0800Z.nc
https://data-dataref.ifremer.fr/marc/f1_e2500_agrif/MARC_F1-MARS3D-SEINE/best_estimate/2018/MARC_F1-MARS3D-SEINE_20180801T0800Z.nc
https://data-dataref.ifremer.fr/marc/f1_e2500_agrif/MARC_F1-MARS3D-SEINE/best_estimate/2018/MARC_F1-MARS3D-SEINE_20180801T0800Z.nc
https://data-dataref.ifremer.fr/marc/f1_e2500_agrif/MARC_F1-MARS3D-SEINE/best_estimate/2018/MARC_F1-MARS3D-SEINE_20180801T0800Z.nc
https://data-dataref.ifremer.fr/marc/f1_e2500_agrif/MARC_F1-MARS3D-SEINE/best_estimate/2018/MARC_F1-MARS3D-SEINE_20180801T0800Z.nc
https://data-dataref.ifremer.fr/marc/f1_e2500_agrif/MARC_F1-MARS3D-SEINE/best_estimate/2018/MARC_F1-MARS3D-SEINE_20180801T0800Z.nc
https://data-dataref.ifremer.fr/marc/f1_e2500_agrif/MARC_F1-MARS3D-SEINE/best_estimate/2018/MARC_F1-MARS3D-SEINE_20180801T0800Z.nc
https://data-dataref.ifremer.fr/marc/f1_e25

Open the translated catalogue and access the data itself (throught https:// access)

repeat this process on DATARMOR, and same the translated kerchunk file at https://data-taos.ifremer.fr/kerchunk_https/test.json

In [12]:
url_out= 'https://data-taos.ifremer.fr/kerchunk_https/test.json'


import xarray as xr
import kerchunk

backend_args= {
        "storage_options": {
            "fo": url_out,
        },
        "consolidated": False
    }
ds=xr.open_mfdataset("reference://", engine="zarr", backend_kwargs=backend_args)
ds.H0.min().compute()

<xarray.DataArray 'H0' ()>
array(-6.8009653, dtype=float32)

Example of opening file direct from a netcdf file it self.  

In [15]:
ds.U

<xarray.DataArray 'U' (time: 1, nj_u: 183, ni_u: 413)>
[75579 values with dtype=float32]
Coordinates:
  * ni_u         (ni_u) float32 0.5 1.5 2.5 3.5 4.5 ... 409.5 410.5 411.5 412.5
  * nj_u         (nj_u) float32 0.0 1.0 2.0 3.0 4.0 ... 179.0 180.0 181.0 182.0
  * time         (time) datetime64[ns] 2018-08-01
    longitude_u  (nj_u, ni_u) float64 ...
    latitude_u   (nj_u, ni_u) float64 ...
Attributes:
    long_name:      barotropic zonal velocity
    standard_name:  barotropic_sea_water_x_velocity_at_u_location
    units:          m s-1
    valid_min:      -20.0
    valid_max:      20.0

In [13]:
import fsspec
import xarray as xr
url='https://data-dataref.ifremer.fr/marc/f1_e2500_agrif/MARC_F1-MARS3D-SEINE/best_estimate/2018/MARC_F1-MARS3D-SEINE_20180801T0000Z.nc'
fs = fsspec.filesystem('https')
ds = xr.open_dataset(fs.open(url))
ds.H0.min().compute()
#xr.open_dataset('https://data-dataref.ifremer.fr/marc/f1_e2500_agrif/MARC_F1-MARS3D-SEINE/best_estimate/2018/MARC_F1-MARS3D-SEINE_20180801T0000Z.nc')

<xarray.DataArray 'H0' ()>
array(-6.8009653, dtype=float32)

# NEXT TEST: compressed meta data set. 

In [5]:
#from DATARMOR
url_in = 'file:///home/datawork-taos-s/public/kerchunk_file/test.json.zst'
#from PC
url_in= 'https://data-taos.ifremer.fr/kerchunk_file/test.json.zst'

d= url_in
import xarray as xr
import kerchunk
backend_args = {
    "consolidated": False,
    "storage_options": {
        "fo": url_in,
        "target_options": {
            "compression": "zstd"
        },
#        "target_protocol": "file"
    }}


#backend_args= {
#        "storage_options": {
#            "fo": url_in,
#        },
#        "consolidated": False
#    }
ds=xr.open_mfdataset("reference://", engine="zarr", backend_kwargs=backend_args)
ds.U.min()#.compute()

<xarray.DataArray 'U' ()>
dask.array<_nanmin_skip-aggregate, shape=(), dtype=float32, chunksize=(), chunktype=numpy.ndarray>

# SOMEHOW here, file not found err for same file??

In [4]:
#DATARMOR
url_out = 'file:///home/datawork-taos-s/public/kerchunk_https/testzst.json'
#PC

url_out = 'file:///Users/todaka/python/git/auto-kerchunk/testzst.json'

renames={'file:///home/ref-marc/':'https://data-dataref.ifremer.fr/marc/'} 

storage_options_in= {
#        "fo": d,
        "target_options": {
            "compression": "zstd"
        },
        "target_protocol": "https"
    }

storage_options_out= {
#        "fo": d,
#        "target_options": {
#            "compression": "zstd"
#        },
        "target_protocol": "file"
    }

rename_target_files(
    url_in, renames, url_out, storage_options_in#=None
    , storage_options_out=None)

in rename_target_files


FileNotFoundError: https://data-taos.ifremer.fr/kerchunk_file/test.json.zst

In [ ]:
cat = intake.open_catalog("/home/datawork-lops-iaocea/catalog/intake/marc.yaml")

In [ ]:
import intake

cat = intake.open_catalog("/home/datawork-lops-iaocea/catalog/intake/marc.yaml")

import xarray as xr

In [ ]:
cat.marc(region="MARC_F1-MARS3D-SEINE", year="2018").to_dask()

In [ ]:
import xarray as xr
import kerchunk
d="file:///home/datawork-lops-iaocea/catalog/kerchunk/ref-marc/MARC_F1-MARS3D-SEINE/2018.json.zst"


backend_args = {
    "consolidated": False,
    "storage_options": {
        "fo": d,
        "target_options": {
            "compression": "zstd"
        },
        "target_protocol": "file"
    }}
print(xr.open_dataset("reference://", engine="zarr", backend_kwargs=backend_args))

In [ ]:
import fsspec
import ujson
with fsspec.open(url_in, **(storage_options_in or {})) as f:
    old = ujson.load(f)

In [ ]:
import kerchunk.hdf
import fsspec
u="file:///home/ref-marc/f1_e2500_agrif/MARC_F1-MARS3D-SEINE/best_estimate/2018/MARC_F1-MARS3D-SEINE_20180801T0000Z.nc"

with fsspec.open(u) as inf:
    h5chunks = kerchunk.hdf.SingleHdf5ToZarr(inf, u, inline_threshold=100)
    chunk_info = h5chunks.translate()

In [ ]:
import fsspec
import kerchunk.hdf
remote_filename = 'https://object-store.cloud.muni.cz/swift/v1/foss4g-data/CGLS_LTS_1999_2019/c_gls_NDVI-LTS_1999-2019-1221_GLOBE_VGT-PROBAV_V3.0.1.nc'
with fsspec.open(remote_filename) as inf:
    h5chunks = kerchunk.hdf.SingleHdf5ToZarr(inf, remote_filename, inline_threshold=100)
    chunk_info = h5chunks.translate()

In [ ]:
import fsspec
import kerchunk.hdf
remote_filename = 'file:///home/ref-marc/f1_e2500_agrif/MARC_F1-MARS3D-SEINE/best_estimate/2018/MARC_F1-MARS3D-SEINE_20180801T0000Z.nc'
with fsspec.open(remote_filename) as inf:
    h5chunks = kerchunk.hdf.SingleHdf5ToZarr(inf, remote_filename, inline_threshold=100)
    chunk_info = h5chunks.translate()

In [ ]:
import auto_kerchunk 

auto_kerchunk

In [ ]:
python -m auto_kerchunk single-hdf5-to-zarr $FILE --glob $GLOB $TMP